In [1]:
!pip install ko-sentence-transformers

    ERROR: Command errored out with exit status 1:
     command: 'C:\Anaconda3\python.exe' -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\admin\\AppData\\Local\\Temp\\pip-install-caouzmzl\\ko-sentence-transformers_6071a109e1c84e9d9504e6075e72bc24\\setup.py'"'"'; __file__='"'"'C:\\Users\\admin\\AppData\\Local\\Temp\\pip-install-caouzmzl\\ko-sentence-transformers_6071a109e1c84e9d9504e6075e72bc24\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base 'C:\Users\admin\AppData\Local\Temp\pip-pip-egg-info-kfa8wqu4'
         cwd: C:\Users\admin\AppData\Local\Temp\pip-install-caouzmzl\ko-sentence-transformers_6071a109e1c84e9d9504e6075e72bc24\
    Complete output (5 lines):
    Traceback (most recent call last):
      File "<string>


  Created wheel for ko-sentence-transformers: filename=ko_sentence_transformers-0.2-py3-none-any.whl size=7539 sha256=0c3459c256dfc0b528171b82fedcf89e04b7e4f7eb60c05a5a8dad7ad6cdf591
  Stored in directory: c:\users\admin\appdata\local\pip\cache\wheels\db\e7\77\a61e2d35160510738d50fe4e3ab2bf79817b93597d45c51cd0
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=0938bc7f2a5cad708b5fa7f555f5209c95afa8ea6edb98a384be2dea0c7d3b5c
  Stored in directory: c:\users\admin\appdata\local\pip\cache\wheels\71\67\06\162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built ko-sentence-transformers sentence-transformers


In [2]:
from sentence_transformers import SentenceTransformer, models
from ko_sentence_transformers.models import KoBertTransformer
word_embedding_model = KoBertTransformer("monologg/kobert", max_seq_length=75)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(), pooling_mode='mean')
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


In [3]:
from sentence_transformers import SentenceTransformer, util
import numpy as np

embedder = SentenceTransformer("jhgan/ko-sbert-sts")

# Corpus with example sentences
corpus = ['한 남자가 음식을 먹는다.',
          '한 남자가 빵 한 조각을 먹는다.',
          '그 여자가 아이를 돌본다.',
          '한 남자가 말을 탄다.',
          '한 여자가 바이올린을 연주한다.',
          '두 남자가 수레를 숲 속으로 밀었다.',
          '한 남자가 담으로 싸인 땅에서 백마를 타고 있다.',
          '원숭이 한 마리가 드럼을 연주한다.',
          '치타 한 마리가 먹이 뒤에서 달리고 있다.']

corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)

# Query sentences:
queries = ['한 남자가 파스타를 먹는다.',
           '고릴라 의상을 입은 누군가가 드럼을 연주하고 있다.',
           '치타가 들판을 가로 질러 먹이를 쫓는다.']

# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
top_k = 5
for query in queries:
    query_embedding = embedder.encode(query, convert_to_tensor=True)
    cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
    cos_scores = cos_scores.cpu()

    #We use np.argpartition, to only partially sort the top_k results
    top_results = np.argpartition(-cos_scores, range(top_k))[0:top_k]

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")

    for idx in top_results[0:top_k]:
        print(corpus[idx].strip(), "(Score: %.4f)" % (cos_scores[idx]))





Query: 한 남자가 파스타를 먹는다.

Top 5 most similar sentences in corpus:
한 남자가 음식을 먹는다. (Score: 0.6154)
한 남자가 빵 한 조각을 먹는다. (Score: 0.5451)
한 여자가 바이올린을 연주한다. (Score: 0.0980)
한 남자가 말을 탄다. (Score: 0.0686)
한 남자가 담으로 싸인 땅에서 백마를 타고 있다. (Score: 0.0572)




Query: 고릴라 의상을 입은 누군가가 드럼을 연주하고 있다.

Top 5 most similar sentences in corpus:
원숭이 한 마리가 드럼을 연주한다. (Score: 0.7253)
치타 한 마리가 먹이 뒤에서 달리고 있다. (Score: 0.1940)
한 여자가 바이올린을 연주한다. (Score: 0.1483)
한 남자가 담으로 싸인 땅에서 백마를 타고 있다. (Score: 0.0471)
한 남자가 음식을 먹는다. (Score: 0.0082)




Query: 치타가 들판을 가로 질러 먹이를 쫓는다.

Top 5 most similar sentences in corpus:
치타 한 마리가 먹이 뒤에서 달리고 있다. (Score: 0.7887)
원숭이 한 마리가 드럼을 연주한다. (Score: 0.3887)
두 남자가 수레를 숲 속으로 밀었다. (Score: 0.1610)
한 남자가 음식을 먹는다. (Score: 0.0739)
그 여자가 아이를 돌본다. (Score: 0.0568)


In [4]:
# Corpus with example sentences
corpus = ['테스트 해봤는데 정말 촉촉하고 피부에 흡수도 잘 되고 마무리감이 좋다.',
          ' 밤에 잘 때 듬뿍은 아니어도 한번 발라주고 그 위에 펴발라주고 잔다.',
          '이제품 너무 마음에 들어서 추천하고 싶다.',
          '그 여자는 화장품을 산다.',
          '아빠한테 화장품을 선물해드릴것이다.',
          '두 남자가 수레를 숲 속으로 밀었다.',
          '나도 이걸 사야지.',
          '원숭이 한 마리가 립스틱을 가져갔다.',
          '치타 한 마리가 먹이 뒤에서 달리고 있다.']

corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)

# Query sentences:
queries = ['얼른 많은 분들이 사용해보셨으면 하는 마음입니다.']

# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
top_k = 5
for query in queries:
    query_embedding = embedder.encode(query, convert_to_tensor=True)
    cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
    cos_scores = cos_scores.cpu()

    #We use np.argpartition, to only partially sort the top_k results
    top_results = np.argpartition(-cos_scores, range(top_k))[0:top_k]

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")

    for idx in top_results[0:top_k]:
        print(corpus[idx].strip(), "(Score: %.4f)" % (cos_scores[idx]))





Query: 얼른 많은 분들이 사용해보셨으면 하는 마음입니다.

Top 5 most similar sentences in corpus:
이제품 너무 마음에 들어서 추천하고 싶다. (Score: 0.3566)
나도 이걸 사야지. (Score: 0.3479)
테스트 해봤는데 정말 촉촉하고 피부에 흡수도 잘 되고 마무리감이 좋다. (Score: 0.3029)
밤에 잘 때 듬뿍은 아니어도 한번 발라주고 그 위에 펴발라주고 잔다. (Score: 0.2757)
원숭이 한 마리가 립스틱을 가져갔다. (Score: 0.2122)


In [1]:
import numpy
print(numpy.__version__)

1.21.6
